In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Conv2DTranspose, Reshape, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [ ]:
# Load dataset
dataset_dir = 'dataset'

In [ ]:
# Get image size from the first image
img = plt.imread(f'{dataset_dir}/Non Demented/OAS1_0001_MR1_mpr-1_100.jpg')

img_height, img_width, _ = img.shape

print(f'Image size: {img_height}x{img_width}')

In [ ]:
# Set batch size
batch_size = 16

In [ ]:
# Split class folders into train test split
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Number of classes
num_classes = len(train_generator.class_indices)

In [ ]:
# Number of epochs
epochs_number = 20

#### Testing pretrained ResNet-50 model


In [ ]:
# Load the ResNet50 model
base_resNet_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_resNet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Create model
resnet_model = Model(inputs=base_resNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_resNet_model.layers:
    layer.trainable = False

# Compile model
optimizer = Adam(learning_rate=0.001)
resnet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train model
resNet_history = resnet_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# Save trained model
resnet_model.save("resNet_model.h5")

In [ ]:
# Plot training and validation accuracy
plt.plot(resNet_history.history["accuracy"], label="Training accuracy")
plt.plot(resNet_history.history["val_accuracy"], label="Validation accuracy")
plt.title("Training and validation accuracy - ResNet50V2")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Testing DenseNet-121 model


In [ ]:
# Load pre-trained DenseNet121 model
base_denseNet_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_denseNet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Create model
denseNet_model = Model(inputs=base_denseNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_denseNet_model.layers:
    layer.trainable = False

# Compile model
optimizer = Adam(learning_rate=0.001)
denseNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train model
denseNet_history = denseNet_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# Save trained model
denseNet_model.save("denseNet_model.h5")

In [ ]:
plt.plot(denseNet_history.history["accuracy"], label="Training accuracy")
plt.plot(denseNet_history.history["val_accuracy"], label="Validation accuracy")
plt.title("Training and validation accuracy - DenseNet121")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Testing custom CNN model


In [ ]:
# Custom CNN architecture
custom_model = Sequential(
    [
        Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ]
)

# Compile the model
optimizer = Adam(learning_rate=0.001)
custom_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train the model
custom_CNN_history = custom_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# Save the trained model
custom_model.save("custom_CNN_model.h5")

In [ ]:
plt.plot(custom_CNN_history.history["accuracy"], label="Training accuracy")
plt.plot(custom_CNN_history.history["val_accuracy"], label="Validation accuracy")
plt.title("Training and validation accuracy - Custom CNN")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Evaluate models

In [ ]:
# Table comparing the performance of the three models
resNet_accuracy = resNet_history.history["accuracy"][-1]
resNet_val_accuracy = resNet_history.history["val_accuracy"][-1]
denseNet_accuracy = denseNet_history.history["accuracy"][-1]
denseNet_val_accuracy = denseNet_history.history["val_accuracy"][-1]
custom_CNN_accuracy = custom_CNN_history.history["accuracy"][-1]
custom_CNN_val_accuracy = custom_CNN_history.history["val_accuracy"][-1]

model_comparison = pd.DataFrame(
    {
        "Model": ["ResNet50V2", "DenseNet121", "Custom CNN"],
        "Train Accuracy": [resNet_accuracy, denseNet_accuracy, custom_CNN_accuracy],
        "Validation Accuracy": [resNet_val_accuracy, denseNet_val_accuracy, custom_CNN_val_accuracy],
    }
)

print(model_comparison)